## Just Check Data

In [1]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import PIL
from mmdet.apis import inference_detector, init_detector, show_result_pyplot
import mmcv

import pickle

In [2]:
dir = 'swin_nancho_pre_kuzh'
model_name = 'swin_nancho.py'
model_load = 'mmdetection/work_dirs/'+dir+'/best_mAP_epoch_15.pth'

root_dir = '/home/mauricio/Documents/Pytorch/mmdetection/data/Nancho_dataset/'
config_model = 'mmdetection/work_dirs/'+dir+'/'+model_name 
save_path = '/home/mauricio/Documents/Pytorch/mmdetection/code/mmdetection/work_dirs/'+dir+'/'

In [4]:
# https://github.com/matterport/Mask_RCNN/issues/2408
# https://gist.github.com/aunsid/b28c87f98983f00163f6e588e3da1191
def get_iou(a, b, epsilon=1e-5, intersection_check=False):
    x1 = max(a[0], b[0])
    y1 = max(a[1], b[1])
    x2 = min(a[2], b[2])
    y2 = min(a[3], b[3])
    score = b[4]
    
    width = (x2 - x1)
    height = (y2 - y1)

    if (width < 0) or (height < 0):
        if intersection_check:
            return 0.0, False
        else:
            return 0.0
    area_overlap = width * height

    area_a = (a[2] - a[0]) * (a[3] - a[1])
    area_b = (b[2] - b[0]) * (b[3] - b[1])
    area_combined = area_a + area_b - area_overlap

    iou = area_overlap / (area_combined + epsilon)
    if intersection_check:
        return iou, bool(area_overlap)
    else:
        return iou

###todo: get mAP after plotting adding the info in the df
def average_precision(recalls, precisions, mode='area'):
    """Calculate average precision (for single or multiple scales).

    Args:
        recalls (ndarray): shape (num_scales, num_dets) or (num_dets, )
        precisions (ndarray): shape (num_scales, num_dets) or (num_dets, )
        mode (str): 'area' or '11points', 'area' means calculating the area
            under precision-recall curve, '11points' means calculating
            the average precision of recalls at [0, 0.1, ..., 1]

    Returns:
        float or ndarray: calculated average precision
    """
    no_scale = False
    if recalls.ndim == 1:
        no_scale = True
        recalls = recalls[np.newaxis, :]
        precisions = precisions[np.newaxis, :]
    assert recalls.shape == precisions.shape and recalls.ndim == 2
    num_scales = recalls.shape[0]
    print("class: ",num_scales )
    ap = np.zeros(num_scales, dtype=np.float32)
    if mode == 'area':
        zeros = np.zeros((num_scales, 1), dtype=recalls.dtype)
        ones = np.ones((num_scales, 1), dtype=recalls.dtype)
        mrec = np.hstack((zeros, recalls, ones))
        mpre = np.hstack((zeros, precisions, zeros))
        for i in range(mpre.shape[1] - 1, 0, -1):
            mpre[:, i - 1] = np.maximum(mpre[:, i - 1], mpre[:, i])
        for i in range(num_scales):
            ind = np.where(mrec[i, 1:] != mrec[i, :-1])[0]
            ap[i] = np.sum(
                (mrec[i, ind + 1] - mrec[i, ind]) * mpre[i, ind + 1])
    elif mode == '11points':
        for i in range(num_scales):
            for thr in np.arange(0, 1 + 1e-3, 0.1):
                precs = precisions[i, recalls[i, :] >= thr]
                prec = precs.max() if precs.size > 0 else 0
                ap[i] += prec
        ap /= 11
    elif mode == '101points':
        for i in range(num_scales):
            for thr in np.arange(0.5, .95 , 0.05):
                precs = precisions[i, recalls[i, :] >= thr]
                prec = precs.max() if precs.size > 0 else 0
                ap[i] += prec
        ap /= 101
    else:
        raise ValueError(
            'Unrecognized mode, only "area" and "11points" are supported')
    if no_scale:
        ap = ap[0]
    return ap

def calc_conditions(id, gt_boxes, pred_boxes, iou_thresh=0.5, hard_fp=True):
    gt_class_ids_ = np.zeros(len(gt_boxes))
    pred_class_ids_ = np.zeros(len(pred_boxes))

    tp, fp, fn = 0, 0, 0
    fppc = det_rate = 0
    for i in range(len(gt_class_ids_)):
        iou = []
        for j in range(len(pred_class_ids_)):
            now_iou, intersect = get_iou(gt_boxes[i], pred_boxes[j], intersection_check=True)
            if now_iou >= iou_thresh and intersect:
                iou.append(now_iou)
                gt_class_ids_[i] = 1
                pred_class_ids_[j] = 1
        if len(iou) > 0: 
            tp += 1 
            fp += len(iou) - 1
    fn += np.count_nonzero(np.array(gt_class_ids_) == 0)
    fp += np.count_nonzero(np.array(pred_class_ids_) == 0)
    
    if tp > 0:
        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
        f1_score = 2 * ((precision * recall) / (precision + recall))
        iou = tp / (tp + fp + fn)
        fppc = fp / (tp+fn) #(tp + fp - (tp))/(tp+fn)
        det_rate= tp / (tp + fn) #accuracy tp/(tp+fp+fn) 
        fppc= float("{0:.5f}".format(fppc)) #False positives per character
        det_rate= float("{0:.5f}".format(det_rate)) #detection rate (accuracy)
        
    else:
        fppc = fp / (tp+fn) #(tp + fp - (tp))/(tp+fn)
        fppc= float("{0:.5f}".format(fppc)) #False positives per character
        precision = recall = f1_score = iou = det_rate  = 0.0

    return {'image_id': id, 'fppc':fppc, 'det rate':det_rate, 'tp':tp, 'fp':fp, 'fn':fn,  
            'precision':precision, 'recall':recall, 'f1':f1_score}

In [10]:
gt_file = open(root_dir+'dtest.pkl', 'rb')
gt_data = pickle.load(gt_file)

load checkpoint from local path: mmdetection/work_dirs/swin_nancho_pre_kuzh/best_mAP_epoch_15.pth


In [20]:
det_thres=  0.2 #0.35
data = []

model = init_detector(config_model, model_load)
for img_p in gt_data:
    img = mmcv.imread(root_dir + 'test_images/'+img_p['filename'])
    pred = inference_detector(model, img)
    data.append(calc_conditions(img_p['filename'],img_p['ann']['bboxes'],pred[1], det_thres))
    # print(pred)
    # print("-----")
    # break
del model 
torch.cuda.empty_cache()

load checkpoint from local path: mmdetection/work_dirs/swin_nancho_pre_kuzh/best_mAP_epoch_15.pth


/usr/local/lib/python3.8/dist-packages/mmdet/datasets/utils.py:66: UserWarning: "ImageToTensor" pipeline is replaced by "DefaultFormatBundle" for batch inference. It is recommended to manually replace it in the test data pipeline in your config file.
  warnings.warn(


In [21]:
df = pd.DataFrame(data)

tp=sum(df['tp'])
fp=sum(df['fp'])
fn=sum(df['fn'])
fppc=fp/(tp+fn)
dr=tp/(tp+fn)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1 = (2 * precision * recall) / (precision + recall)
ap_101= average_precision(df['recall'],df['precision'],'101points')

new= {'image_id':'promedio', 'fppc':fppc, 'det rate':dr, 'tp':tp, 'fp':fp, 'fn':fn, 
        'precision':precision, 'recall':recall, 'f1':f1, 'mAP101':ap_101}
df= df.append(new, ignore_index = True)
df

class:  1


/tmp/ipykernel_17470/1722196695.py:47: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  recalls = recalls[np.newaxis, :]
/tmp/ipykernel_17470/1722196695.py:48: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  precisions = precisions[np.newaxis, :]


,image_id,fppc,det rate,tp,fp,fn,precision,recall,f1,mAP101
0,test_1.jpg,0.026040,0.984380,189,5,3,0.974227,0.984375,0.979275,NaN
1,test_24.jpg,0.032030,0.957300,269,9,12,0.967626,0.957295,0.962433,NaN
2,test_53.jpg,0.040820,0.959180,188,8,8,0.959184,0.959184,0.959184,NaN
3,test_75.jpg,0.115150,0.903030,149,19,16,0.886905,0.903030,0.894895,NaN
4,test_48.jpg,0.032790,0.978140,179,6,4,0.967568,0.978142,0.972826,NaN
5,test_61.jpg,0.083330,1.000000,12,1,0,0.923077,1.000000,0.960000,NaN
6,test_73.jpg,0.092590,1.000000,54,5,0,0.915254,1.000000,0.955752,NaN
7,test_8.jpg,0.035970,0.971220,135,5,4,0.964286,0.971223,0.967742,NaN
8,test_21.jpg,0.029780,0.900740,363,12,40,0.968000,0.900744,0.933162,NaN
9,test_62.jpg,0.428570,1.000000,14,6,0,0.700000,1.000000,0.823529,NaN


In [22]:
ap_area= average_precision(df['recall'],df['precision'],'area')
ap_11= average_precision(df['recall'],df['precision'],'11points')
ap_101= average_precision(df['recall'],df['precision'],'101points')
print('mAP_Area:', ap_area)
print('mAP11:', ap_11)
print('mAP101:', ap_101)

class:  1
class:  1
class:  1
mAP_Area: 0.9199281
mAP11: 0.9847326
mAP101: 0.0880288


/tmp/ipykernel_17470/1722196695.py:47: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  recalls = recalls[np.newaxis, :]
/tmp/ipykernel_17470/1722196695.py:48: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  precisions = precisions[np.newaxis, :]


In [153]:
calc_conditions(img_p['filename'], img_p['ann']['bboxes'], pred[1], 0.1)

{'image_id': 'test_46.jpg',
 'fppc': 0.0597,
 'det rate': 0.98507,
 'tp': 198,
 'fp': 12,
 'fn': 3,
 'precision': 0.9428571428571428,
 'recall': 0.9850746268656716,
 'f1': 0.9635036496350364}

### Save data??

In [20]:
df.to_csv(save_path+str(det_thres)+"_statistics.csv", index=False)

In [21]:
for img_p in gt_data:
    img = mmcv.imread(root_dir + 'test_images/'+img_p['filename'])
    pred = inference_detector(model, img)
    labs = np.ones(len(pred[1]),dtype=str)
    # data.append(calc_conditions(img_p['filename'],img_p['ann']['bboxes'],pred[1], det_thres))
    image = mmcv.visualization.imshow_bboxes(img,img_p['ann']['bboxes'],thickness=2, show=False)
    mmcv.visualization.imshow_det_bboxes(image, pred[1], labels= labs, thickness=2, score_thr= det_thres,
                                        bbox_color='red',text_color='red',font_scale=0.02, 
                                        out_file=save_path+str(det_thres)+'_shows/'+img_p['filename'],show=False)
    # print("-----")
torch.cuda.empty_cache()

In [113]:
img_p['ann']['bboxes_ignore']

array([], shape=(0, 4), dtype=float32)

In [154]:
from mmdet.core.evaluation.mean_ap import tpfp_default
tpp, fpp = tpfp_default(pred[1],img_p['ann']['bboxes'],img_p['ann']['bboxes_ignore'], iou_thr=0.35)
print(sum(sum(tpp)), sum(sum(fpp)))

195.0 8.0


In [19]:
# save predictions
#pred
#mmcv.dump(pred, save_path+'predic.pkl')